# Dimensionality Reduction
## 维数灾难
##### 例如，在单位正方形内随机选点，那么随机选的点离所有边界大于0.001的概率为0.4%，但在一个10000维单位超立方体，这个概率大于99 999999%。因此高维超立方体中的大多数随机点都非常接近边界。
##### 在单位超立方体随机选两个点，在2维空间，两个点之间平均距离约为0.52；在3维空间约为0.66，在1,000,000维空间，则约为408.25，因此在高维空间，这很可能会使得训练集和测试集相差很大，导致训练过拟合。
##### 理论上，只要通过增加训练集，就能达到训练空间足够密集。但是随着维度的增加，需要的训练集是呈指数增长的。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# PCA

In [ ]:
# PCA 基础, SVD 分解
# numpy
x1 = np.random.randn(100)
x2 = x1 + np.random.rand(100)
X = np.stack([x1, x2], axis=1)
X_centered = X - np.mean(X, axis=1, keepdims=True)  # PCA假设数据以原点为中心，Scikit-learn中的PCA类已经减去均值
u, s, v = np.linalg.svd(X_centered)
d = 1
X_D = X_centered.dot(v.T[:, :d])  # 降维后数据

# sklearn
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X_D = pca.fit_transform(X)
pca.components_, pca.explained_variance_ratio_

In [ ]:
# 选择合理的维数
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum <= 0.95) + 1

pca = PCA(n_components=0.95)

In [ ]:
# 增量PCA（IPCA）
from sklearn.datasets import fetch_mldata
digits = fetch_mldata("MNIST original")
X = digits["data"]
# IncrementalPCA
from sklearn.decomposition import IncrementalPCA
inc_pca = IncrementalPCA(n_components=154)
n_batches = 100
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)
X_D = inc_pca.transform(X)
# memmap, batch_size
def helper(filename, m, n, n_batches):
    X = np.memmap(filename, mode="r", dtype=np.float64, shape=(m, n))
    batch_size = m // n_batches
    inc_pca = IncrementalPCA(n_components=154, batch_size = batch_size)
    inc_pca.fit(X)

# 随机PCA
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_D = rnd_pca.fit_transform(X)

# 核PCA(Kernel PCA)

In [ ]:
from sklearn.datasets import make_swiss_roll
from mpl_toolkits.mplot3d import Axes3D
X, t = make_swiss_roll(n_samples=1000, noise=0.2)
fig = plt.figure(figsize=(15,6))
fig.add_subplot(1, 2, 1, projection='3d')
plt.scatter(X[:,0], X[:,1], X[:,2], c=t)

from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_D = kpca.fit_transform(X)

fig.add_subplot(1, 2, 2)
plt.scatter(X_D[:,0], X_D[:,1], c=t)
# 此方法要使用大量内存，可能会使内存溢出

In [ ]:
# grid 选择合适的核与参数
from sklearn.datasets import fetch_mldata
digits = fetch_mldata("MNIST original")
X, y = digits["data"][:8000].astype(np.float64), digits["target"][:8000]
# X, y = digits["data"].astype(np.float64), digits["target"]

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
clf = Pipeline([
    ("std", StandardScaler()),
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression(solver="lbfgs", C=2, multi_class="multinomial"))
])
param_grid = [
    {"kpca__kernel": ["rbf", "sigmoid"], "kpca__gamma": np.linspace(0.001, 0.5, 50)}
]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)
print(grid_search.best_params_)

# 局部线性嵌入（Locally Linear Embedding）LLE

In [ ]:
from sklearn.datasets import make_swiss_roll
X, y = make_swiss_roll(1000, noise=0.0)
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(121, projection="3d")
ax.scatter(X[:,0], X[:,1], X[:,2], c=y)
# LocallyLinearEmbedding
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_neighbors=10, n_components=2, n_jobs=-1)
X_D = lle.fit_transform(X)

ax = fig.add_subplot(122)
ax.scatter(X_D[:,0], X_D[:,1], c=y)